In [1]:
# 최적화 코드
# light gbm 사용하기
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import RobustScaler

dataset = pd.read_csv('train.csv')
X = dataset.iloc[:,1:57]
y = dataset.iloc[:,57:]

X.drop(['X_04', 'X_10', 'X_11', 'X_23', 'X_47', 'X_48', 'X_37'], axis = 1, inplace = True) 





# 훈련데이터, 테스트 데이터 분리
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)



# 평가방식
from sklearn import metrics
def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(0,14): # ignore 'ID'
        rmse = metrics.mean_squared_error(gt[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:14])
    return score

In [5]:
# lightgbm 옵션 변경해보기
test = pd.read_csv('test.csv')
test.drop(['X_04', 'X_10', 'X_11', 'X_23', 'X_47', 'X_48', 'X_37', 'ID'], axis = 1, inplace = True) 
from lightgbm import LGBMRegressor
import lightgbm
lgb_reg = MultiOutputRegressor(LGBMRegressor(n_estimators = 400, learning_rate = 0.02)).fit(X, y)
lgb_preds = lgb_reg.predict(test)



In [26]:
print(X.shape, test.shape)

(39608, 49) (39608, 49)


In [6]:
import xgboost
from xgboost import XGBRegressor
xgb_reg = MultiOutputRegressor(XGBRegressor(n_estimators = 400, learning_rate = 0.02)).fit(X, y)
xgb_preds = xgb_reg.predict(test)


C:\Users\kyeongtae\Anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
C:\Users\kyeongtae\Anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [7]:
# 랜덤 포레스트
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
rdm = MultiOutputRegressor(RandomForestRegressor(n_estimators = 100, min_samples_leaf = 8) ).fit(X, y) #이거는 한 20분걸림 
preds_rdm = rdm.predict(test)


In [8]:
pred_mean=np.zeros((lgb_preds.shape))
for j in range(14): # 열개수
    for i in range(len(xgb_preds)): #행개수
        avg=np.mean([xgb_preds[i][j],preds_rdm[i][j],lgb_preds[i][j]]) # 각 예측값의 평균을 구함
        pred_mean[i][j]=avg # 구한 평균을 해당 위치에 저장
pred_mean

submit = pd.read_csv('sample_submission.csv')
for idx, col in enumerate(submit.columns):
    if col=='ID':
        continue
    submit[col] = pred_mean[:,idx-1]
submit.to_csv('D:\\구글 드라이브 백업\\자율주행/submit0817_1.csv', index = False)

In [86]:
submit


,ID,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,TEST_00001,1.428897,1.218594,1.115950,14.212423,31.306828,16.708520,3.158141,-26.131583,-26.173738,-22.257278,24.452153,-26.128253,-26.043431,-26.069677
1,TEST_00002,1.464256,1.185099,1.143468,13.508518,31.109948,16.649687,3.141336,-26.149595,-26.163902,-22.277700,24.405431,-26.112950,-26.110327,-26.131393
2,TEST_00003,1.403446,1.121835,1.085505,14.858107,32.226092,16.816509,3.072211,-25.971477,-26.030619,-22.187581,24.611305,-25.884619,-25.911703,-26.008009
3,TEST_00004,1.405865,1.110401,1.034866,15.005969,32.293835,17.034210,3.098526,-25.665267,-25.735830,-21.789505,24.819393,-25.673316,-25.634063,-25.683355
4,TEST_00005,1.317484,0.980787,0.941239,15.049888,31.726223,16.964131,3.126211,-25.702749,-25.686826,-22.023329,24.715723,-25.660345,-25.654100,-25.680237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39603,TEST_39604,1.243033,0.957183,1.003499,13.431101,31.060184,16.727172,3.175229,-26.532940,-26.518066,-22.867469,24.348340,-26.426037,-26.415819,-26.433552
39604,TEST_39605,1.236867,0.906577,0.963341,14.051220,31.090127,16.739514,3.180113,-26.395509,-26.382695,-22.831486,24.409514,-26.321461,-26.332119,-26.338651
39605,TEST_39606,1.232326,0.892477,0.950580,13.044974,31.337071,16.628472,3.191654,-26.473860,-26.480870,-22.822834,24.263007,-26.453275,-26.434446,-26.439164
39606,TEST_39607,1.221571,0.867143,0.924468,13.539953,31.096282,16.733000,3.184672,-26.474144,-26.480679,-22.774455,24.511125,-26.423190,-26.413167,-26.432784


In [41]:
test = sc.transform(test);test

array([[ 68.3600677 , 103.57387341,  76.94547335, ..., 129.53322309,
        132.74234743, 121.65776635],
       [ 67.34319554, 103.57287099,  69.6198891 , ..., 132.2294584 ,
        133.2318017 , 123.97281441],
       [ 69.37793778, 103.57287099,  69.21848723, ..., 129.87296099,
        127.87189808, 114.28610001],
       ...,
       [ 69.37793778, 103.57287099,  63.89991236, ..., 125.9023126 ,
        135.25281938, 127.4594394 ],
       [ 67.34319554, 103.57387341,  61.99325345, ..., 129.1192094 ,
        132.27962262, 131.66415747],
       [ 71.41268001, 103.57287099,  63.29780955, ..., 131.57680832,
        139.48732981, 121.27791177]])

In [80]:
test = sc.transform(test)

In [81]:
test

array([[ 0.04055377,  2.2600073 ,  1.50451873, ...,  0.29172007,
        -0.6973233 , -1.14795976],
       [-0.34208937, -0.44247645,  0.10221084, ...,  0.75854938,
        -0.62539339, -0.74582908],
       [ 0.42357242, -0.44247645,  0.02537205, ...,  0.35054268,
        -1.41308158, -2.42843995],
       ...,
       [ 0.42357242, -0.44247645, -0.9927419 , ..., -0.33694   ,
        -0.32838586, -0.14019203],
       [-0.34208937,  2.2600073 , -1.35772614, ...,  0.22003726,
        -0.76532505,  0.59017991],
       [ 1.18923421, -0.44247645, -1.10800008, ...,  0.64554879,
         0.29391523, -1.21394162]])

In [5]:
cor = dataset.corr()

In [55]:
result = []

for i in range(9):
    print(f'Y_0{i+1}의 상관관계')
    cor = np.abs(cor)
    cor = cor.sort_values(by = f'Y_0{i+1}')
    print(cor.iloc[:10, i])
    result.append(cor.index[:10])
    
    
    
for i in range(9,14,1):
    print(f'Y_{i+1}의 상관관계')
    cor = np.abs(cor)
    cor = cor.sort_values(by = f'Y_{i+1}')
    print(cor.iloc[:10, i])
    result.append(cor.index[:10])

Y_01의 상관관계
X_37    0.000310
X_19    0.000464
X_34    0.000769
X_35    0.001088
X_44    0.001202
X_05    0.003528
X_36    0.004959
X_02    0.005138
X_32    0.005400
X_41    0.005500
Name: Y_01, dtype: float64
Y_02의 상관관계
X_34    0.000120
X_26    0.001188
X_41    0.002081
X_37    0.002178
X_24    0.002228
X_30    0.002918
X_35    0.003449
X_36    0.003896
X_55    0.004229
X_28    0.004690
Name: Y_02, dtype: float64
Y_03의 상관관계
X_27    0.000027
X_36    0.000725
X_37    0.001247
X_17    0.001841
X_34    0.002251
X_02    0.002252
X_07    0.003445
X_35    0.004185
X_30    0.004941
X_01    0.005171
Name: Y_03, dtype: float64
Y_04의 상관관계
X_09    0.000510
X_02    0.001451
X_31    0.003088
X_07    0.003451
X_36    0.003665
X_46    0.004299
X_11    0.005488
X_34    0.005848
X_10    0.008202
X_15    0.008849
Name: Y_04, dtype: float64
Y_05의 상관관계
X_45    0.000161
X_41    0.000212
X_05    0.001100
X_36    0.001289
X_08    0.001379
X_43    0.001628
X_42    0.001648
X_03    0.002662
X_07    0.002906
X_44

In [16]:
np.abs(cor)

,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,X_09,X_10,...,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
X_08,0.007622,0.001676,0.007182,NaN,0.005577,0.023617,0.054574,1.000000,0.292295,0.006110,...,0.001379,0.000897,0.005572,0.004640,0.005407,0.008864,0.008115,0.005884,0.007206,0.007067
X_17,0.208992,0.024742,0.055498,NaN,0.146094,0.044170,0.007131,0.017170,0.125027,0.013016,...,0.039394,0.033714,0.038919,0.033897,0.026246,0.100202,0.063761,0.030977,0.030848,0.029330
X_18,0.116566,0.007125,0.057195,NaN,0.108166,0.019124,0.011969,0.015341,0.112382,0.009856,...,0.028224,0.009811,0.024976,0.066181,0.058836,0.143861,0.013251,0.062530,0.063037,0.062372
X_15,0.215895,0.017824,0.119012,NaN,0.145811,0.078249,0.001101,0.005391,0.084523,0.007973,...,0.005380,0.007081,0.008189,0.001516,0.003206,0.063742,0.028604,0.002440,0.001517,0.002210
X_19,0.297272,0.038952,0.424348,NaN,0.187890,0.203808,0.004611,0.039801,0.076478,0.008780,...,0.019175,0.063676,0.093032,0.117591,0.106915,0.106221,0.012148,0.109676,0.110775,0.106770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X_09,0.033579,0.002720,0.065713,NaN,0.009923,0.029563,0.012528,0.292295,1.000000,0.013760,...,0.022068,0.015840,0.003295,0.003954,0.003343,0.042074,0.002887,0.003972,0.005898,0.004932
X_04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X_23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X_47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
cor.columns[56]

'Y_01'

In [38]:
cor = dataset.corr().iloc[:,56:]

In [41]:
cor = np.abs(cor.iloc[:56,])

In [56]:
results = []
for i in result:
    for j in i:
        results.append(j)

In [57]:
pd.value_counts(results)

X_36    12
X_37    10
X_35    10
X_08     8
X_46     8
X_09     8
X_34     7
X_15     6
X_11     5
X_07     5
X_49     5
X_54     5
X_02     5
X_56     4
X_05     3
X_28     3
X_41     3
X_55     3
X_27     2
X_26     2
X_44     2
X_12     2
X_30     2
X_45     2
X_39     2
X_42     2
X_29     2
X_25     1
X_32     1
X_51     1
X_43     1
X_03     1
X_17     1
X_24     1
X_10     1
X_31     1
X_19     1
X_01     1
X_40     1
dtype: int64